In [97]:
import csv
import json
import math
from math import sqrt

def csv_reader(path):
    with open(path) as csv_data:
        data = list(csv.reader(csv_data, delimiter=',', skipinitialspace=True))
    return data


#средняя оценка
def avg(data):
    rated = list(filter(lambda x: x != -1, data))
    avg=sum(rated)/len(rated)
    return avg

#упорядоченная метрика сходства
def similarity(userU, data):
    userU_ratings = data[userU]
    sim={}
    for userV in range(1,len(data)):
        if userV == userU:
            continue   
        userV_ratings = data[userV]
        UV, userU_norm, userV_norm = 0, 0, 0
        
        for i in range(len(userU_ratings)):
            if userU_ratings[i] == -1 or userV_ratings[i] == -1:
                continue
            #числитель
            UV += userU_ratings[i] * userV_ratings[i]
            #знаменатели
            userU_norm += userU_ratings[i] * userU_ratings[i]
            userV_norm += userV_ratings[i] * userV_ratings[i]
            sim[userV] = round(UV / (sqrt(userU_norm)*sqrt(userV_norm)), 3)
    sim = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    return sim
    
#рассчет оценки
def getExpRating(userU, data, k):
    avgU = avg(data[userU])
    expRatings = {}
    userU_ratings = data[userU]
    simList=similarity(userU, data); 
    mov_names = csv_reader("movie_names.csv")
    #для каждого фильма, который еще не посмотрел пользователь
    for mov in range(len(userU_ratings)):
        count, num, denom = 0,0,0
        if userU_ratings[mov] == -1:
            #берем первых k похожих
            for i in range(0,k):
                userV = simList[i] #пара (userId, sim)
                if data[userV[0]][mov] != -1:
                    simVU = userV[1]
                    rV = avg(data[userV[0]])
                    rVI = data[userV[0]][mov]
                    num += simVU * (rVI - rV) #числитель
                    denom += abs(simVU) #знаменатель
            rating = round(avgU + num/denom, 3)
            expRatings[str(mov_names[mov][1])] = rating
    return expRatings

"""
выбираем у нескольких похожих пользователей фильмы, которые они смотрели дома в выходные
затем считаем среднюю оценку
выбираем фильмы, которые пользователь еще не смотрел и находим среди них наивысший рейтинг
"""

def getWkndFilms(context_day, context_place, data, userID):
    simList = similarity(userID, data) # отсчитывает ID с 0
    #print(simList)
    films = {}
    for mov in range(len(data[userID])):
        ratings = []
        for i in range(0,k):
            userV = simList[i] #пара (userId, sim)
            if data[userV[0]-1][mov]!=-1 and context_place[userV[0]][mov+1] == "h" and context_day[userV[0]][mov+1] == "Sun" or context_day[userV[0]][mov+1] == "Sat":
                ratings.append(data[userV[0]-1][mov])
        if len(ratings) > 0:
            films[mov] = round(sum(map(int, ratings))/len(ratings), 3)
    return films

    

def getResult(context_day, context_place, data, userID):
    wkndFilms = getWkndFilms(context_day, context_place, data, userID)
    #print(wkndFilms)
    expRating = getExpRating(userID, data, k)
    mov_names = csv_reader("movie_names.csv")
    recMovie = {}
    for mov in wkndFilms:
        if data[userID][mov] == -1:
            recMovie[str(mov_names[mov][1])] = expRating[str(mov_names[mov][1])] 
    #recMovieS = sorted(recMovie.items(), key=lambda x: x[1], reverse=True)
    maxRating = 0
    recMov = ""
    for key, value in recMovie.items():
        if value >= maxRating:
            recMov = key
            recMovRating = value
            
    result = { 
        "user": userID + 1,
        "1": expRating,
        "2": {recMov: recMovRating}
    }
    return result


In [98]:
context_day = []
context_day = csv_reader("context_day.csv")
context_place = []
context_place = csv_reader("context_place.csv")
data = []
for _ in csv_reader("data.csv")[1:]:
    data.append(list(map(int, _[1:])))

k = 4
variant = 31 
userID = variant - 1  

result = getResult(context_day, context_place, data, userID)
print(json.dumps(result, indent=4))

json_file = open("output.json",'w')
json_file.write(json.dumps(result, indent=4))
json_file.close()

{
    "user": 31,
    "1": {
        "Star Wars: Episode V - The Empire Strikes Back": 1.274,
        "Goodfellas": 2.499,
        "The Green Mile": 4.177
    },
    "2": {
        "The Green Mile": 4.177
    }
}
